In [16]:
import pandas as pd
import psutil
import time
import threading

In [17]:
max_cpu_usage = 0
max_ram_usage = 0
max_gpu_usage = 0

def monitor_resources():
    global max_cpu_usage, max_ram_usage
    while True:
        cpu_usage = psutil.cpu_percent(interval=0.5)
        ram_usage = psutil.virtual_memory().used / (1024 ** 2)
        max_cpu_usage = max(max_cpu_usage, cpu_usage)
        max_ram_usage = max(max_ram_usage, ram_usage)
        time.sleep(1)

monitor_thread = threading.Thread(target=monitor_resources, daemon=True)
monitor_thread.start()

From this point onward, we will only train the models on the 30s data

In [18]:
# For time and resource measurement purposes
import time
import psutil

def get_cpu_usage():
    return psutil.cpu_percent(interval=1)

we will first test only the mfcc coefs, let's start by generariong the MFCCs 

In [22]:
mfcc_df = pd.read_csv("/kaggle/working/gtzan_30sec_mfcc_features.csv")

In [23]:
mfcc_df.columns

Index(['mfcc1_mean', 'mfcc1_var', 'mfcc2_mean', 'mfcc2_var', 'mfcc3_mean',
       'mfcc3_var', 'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean', 'mfcc5_var',
       'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean', 'mfcc7_var', 'mfcc8_mean',
       'mfcc8_var', 'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean', 'mfcc10_var',
       'mfcc11_mean', 'mfcc11_var', 'mfcc12_mean', 'mfcc12_var', 'mfcc13_mean',
       'mfcc13_var', 'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean', 'mfcc15_var',
       'mfcc16_mean', 'mfcc16_var', 'mfcc17_mean', 'mfcc17_var', 'mfcc18_mean',
       'mfcc18_var', 'mfcc19_mean', 'mfcc19_var', 'mfcc20_mean', 'mfcc20_var',
       'label'],
      dtype='object')

In [24]:
from scipy.stats import f_oneway

columns = mfcc_df.drop(columns=['label']).columns

for column in columns:
    

# Example: ANOVA for 'rms_mean' across groups in 'label'
    grouped_data = [group[column] for _, group in df.groupby("label")]
    anova_result = f_oneway(*grouped_data)
    print(f"{column} ANOVA F-statistic: {anova_result.statistic}, p-value: {anova_result.pvalue}")


mfcc1_mean ANOVA F-statistic: 130.37183542543974, p-value: 2.494661015032167e-161
mfcc1_var ANOVA F-statistic: 21.51234828075879, p-value: 1.726329448644408e-33
mfcc2_mean ANOVA F-statistic: 83.189909207364, p-value: 1.0539227277289052e-114
mfcc2_var ANOVA F-statistic: 22.22706092522391, p-value: 1.296930574459949e-34
mfcc3_mean ANOVA F-statistic: 39.99480612844631, p-value: 5.049210410470474e-61
mfcc3_var ANOVA F-statistic: 24.10080254855272, p-value: 1.5431020296757826e-37
mfcc4_mean ANOVA F-statistic: 83.86855465766176, p-value: 1.8877433855052627e-115
mfcc4_var ANOVA F-statistic: 61.487769845073075, p-value: 2.3055523149141784e-89
mfcc5_mean ANOVA F-statistic: 32.89918842047562, p-value: 7.940594643409526e-51
mfcc5_var ANOVA F-statistic: 52.855234881086304, p-value: 2.0691428284015633e-78
mfcc6_mean ANOVA F-statistic: 66.03812293703791, p-value: 6.326860348888946e-95
mfcc6_var ANOVA F-statistic: 56.71708165371829, p-value: 2.231637274456634e-83
mfcc7_mean ANOVA F-statistic: 43.2134

training an svm model with hyperparams finetuning using BayesianOptimization

the mapping that we will use

In [25]:
from sklearn.preprocessing import LabelEncoder

# Assuming df['label'] contains the labels
y = mfcc_df['label']

# Initialize the LabelEncoder
encoder = LabelEncoder()

# Fit and transform the labels
y_encoded = encoder.fit_transform(y)

# Map the numbers back to their original labels
label_mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))

print("Encoded Labels:")
print(label_mapping)


Encoded Labels:
{0: 'blues', 1: 'classical', 2: 'country', 3: 'disco', 4: 'hiphop', 5: 'jazz', 6: 'metal', 7: 'pop', 8: 'reggae', 9: 'rock'}


In [26]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay
from bayes_opt import BayesianOptimization
import numpy as np
import matplotlib.pyplot as plt

# Prétraitement des données
X = mfcc_df.drop(columns=['label'])  # Supprimez les colonnes non pertinentes
y = mfcc_df['label']

# Encoder la variable cible
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Normaliser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Fonction d'évaluation pour BayesianOptimization
def optimize_svm(C, gamma):
    model = SVC(C=C, gamma=gamma, kernel='rbf', probability=True, random_state=42)
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

# Définir les limites des hyperparamètres
param_bounds = {
    'C': (0.1, 100),      # Paramètre de régularisation
    'gamma': (1e-4, 1e-1) # Coefficient pour le noyau RBF
}

# Lancer l'optimisation
optimizer = BayesianOptimization(f=optimize_svm, pbounds=param_bounds, random_state=42)
start_opt = time.time_ns()
optimizer.maximize(init_points=5, n_iter=100)
end_opt = time.time_ns()
opt_time = (end_opt - start_opt) / 1e9  # Convert nanoseconds to seconds

print(f"Hyperparameters tuning time: {opt_time:.3f} seconds")


# Extraire les meilleurs hyperparamètres
best_params = optimizer.max
print("Meilleurs hyperparamètres :", best_params)

# Entraîner le modèle avec les meilleurs hyperparamètres
best_model_30sec_mfcc_only = SVC(C=best_params['params']['C'], gamma=best_params['params']['gamma'], kernel='rbf', probability=True, random_state=42)
# Training time
start_train = time.time_ns()
best_model_30sec_mfcc_only.fit(X_train, y_train)
end_train = time.time_ns()
train_time = (end_train - start_train) / 1e9  # Convert nanoseconds to seconds
print(f"Training time: {train_time:.3f} seconds")


# Test (prediction) time
start_test = time.time_ns()
y_pred = best_model_30sec_mfcc_only.predict(X_test)
end_test = time.time_ns()
test_time = (end_test - start_test) / 1e9  # Convert nanoseconds to seconds
print(f"Test (prediction) time: {test_time:.3f} seconds")
print(f"CPU usage during test: {cpu_usage_before}% -> {cpu_usage_after}%")


print("Rapport de classification :\n", classification_report(y_test, y_pred, target_names=encoder.classes_))

# Matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matrice de confusion :\n", conf_matrix)

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
| 1         | 0.67      | 37.52     | 0.09508   |
| 2         | 0.71      | 73.23     | 0.05991   |
| 3         | 0.69      | 15.69     | 0.01568   |
| 4         | 0.685     | 5.903     | 0.08663   |
| 5         | 0.705     | 60.15     | 0.07084   |
| 6         | 0.69      | 89.36     | 0.08234   |
| 7         | 0.69      | 72.15     | 0.08094   |
| 8         | 0.725     | 10.23     | 0.05221   |
| 9         | 0.69      | 0.6638    | 0.04643   |
| 10        | 0.7       | 66.79     | 0.06425   |
| 11        | 0.72      | 73.23     | 0.05736   |
| 12        | 0.72      | 73.22     | 0.05165   |
| 13        | 0.745     | 1.96      | 0.04389   |
| 14        | 0.735     | 1.971     | 0.03947   |
| 15        | 0.685     | 69.05     | 0.01233   |
| 16        | 0.74      | 1.956     | 0.05938   |
| 17        | 0.74      | 1.969     | 0.05439   |
| 18        | 0.715     | 73.24     | 0.04501   |


# Audio Features added

to improve the accuracy we will add more features:

## 1. **Chroma Features**
- **`chroma_stft_mean`**: The mean value of the chroma short-time Fourier transform (STFT). It represents the distribution of energy across different pitch classes.
- **`chroma_stft_var`**: The variance of the chroma STFT, measuring fluctuations in pitch class distribution.

## 2. **Root Mean Square (RMS) Energy**
- **`rms_mean`**: The mean RMS energy of the audio signal, indicating the average loudness.
- **`rms_var`**: The variance of RMS energy, showing variations in loudness.

## 3. **Spectral Features**
- **`spectral_centroid_mean`**: The mean spectral centroid, representing the center of mass of the spectrum and correlating with the perceived brightness of sound.
- **`spectral_centroid_var`**: The variance of the spectral centroid, indicating variations in brightness over time.
- **`spectral_bandwidth_mean`**: The mean spectral bandwidth, measuring the width of the spectral distribution.
- **`spectral_bandwidth_var`**: The variance of spectral bandwidth, showing changes in frequency spread.
- **`rolloff_mean`**: The mean spectral roll-off, representing the frequency below which a certain percentage of total spectral energy is contained.
- **`rolloff_var`**: The variance of spectral roll-off, indicating variations in spectral energy distribution.

## 4. **Zero Crossing Rate (ZCR)**
- **`zero_crossing_rate_mean`**: The mean rate at which the audio signal crosses zero amplitude, related to the noisiness of the sound.
- **`zero_crossing_rate_var`**: The variance of the zero-crossing rate, showing fluctuations in noisiness.

## 5. **Harmonic and Perceptual Features**
- **`harmony_mean`**: The mean harmonic content of the signal, which reflects tonal stability.
- **`harmony_var`**: The variance of harmony, indicating changes in tonal stability over time.
- **`perceptr_mean`**: The mean perceptual sharpness, a psychoacoustic feature related to how sharp or smooth a sound is perceived.
- **`perceptr_var`**: The variance of perceptual sharpness, showing fluctuations in perceived sharpness.

## 6. **Tempo**
- **`tempo`**: The estimated tempo of the audio in beats per minute (BPM), representing the speed of the rhythm.

Each of these features is essential for analyzing and classifying audio signals in machine learning models.


In [27]:
from scipy.stats import f_oneway

coeff_df = pd.read_csv("/kaggle/working/gtzan_30sec_coeff_features.csv")

columns = coeff_df.columns

for column in columns:
    

# Example: ANOVA for 'rms_mean' across groups in 'label'
    grouped_data = [group[column] for _, group in df.groupby("label")]
    anova_result = f_oneway(*grouped_data)
    print(f"{column} ANOVA F-statistic: {anova_result.statistic}, p-value: {anova_result.pvalue}")

chroma_stft_mean ANOVA F-statistic: 176.45328235861118, p-value: 7.683149829866094e-199
chroma_stft_var ANOVA F-statistic: 64.10383258045962, p-value: 1.4065551452115682e-92
rms_mean ANOVA F-statistic: 74.19465163447585, p-value: 1.4859670070126082e-104
rms_var ANOVA F-statistic: 68.27643848679423, p-value: 1.3070953105914825e-97
spectral_centroid_mean ANOVA F-statistic: 97.48492445375801, p-value: 6.437171334336293e-130
spectral_centroid_var ANOVA F-statistic: 82.13464835310555, p-value: 1.5461880467784952e-113
spectral_bandwidth_mean ANOVA F-statistic: 116.60187897974004, p-value: 9.98275171430298e-149
spectral_bandwidth_var ANOVA F-statistic: 29.580172163778805, p-value: 6.7154684261443235e-46
rolloff_mean ANOVA F-statistic: 110.8713172445283, p-value: 2.9390324749054825e-143
rolloff_var ANOVA F-statistic: 52.06524490427322, p-value: 2.2169019491470272e-77
zero_crossing_rate_mean ANOVA F-statistic: 58.716379580769285, p-value: 6.611053380870471e-86
zero_crossing_rate_var ANOVA F-sta

In [ ]:
df = pd.concat([mfcc_df.drop(columns=['label']), coeff_df], axis=1)

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay
from bayes_opt import BayesianOptimization
import numpy as np
import matplotlib.pyplot as plt

# Prétraitement des données
X = df.drop(columns=['label'])  # Supprimez les colonnes non pertinentes
y = df['label']

# Encoder la variable cible
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Normaliser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Fonction d'évaluation pour BayesianOptimization
def optimize_svm(C, gamma):
    model = SVC(C=C, gamma=gamma, kernel='rbf', probability=True, random_state=42)
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

# Définir les limites des hyperparamètres
param_bounds = {
    'C': (0.1, 100),      # Paramètre de régularisation
    'gamma': (1e-4, 1e-1) # Coefficient pour le noyau RBF
}

# Lancer l'optimisation
optimizer = BayesianOptimization(f=optimize_svm, pbounds=param_bounds, random_state=42)
start_opt = time.time_ns()
optimizer.maximize(init_points=5, n_iter=100)
end_opt = time.time_ns()
opt_time = (end_opt - start_opt) / 1e9  # Convert nanoseconds to seconds

print(f"Hyperparameters tuning time: {opt_time:.3f} seconds")


# Extraire les meilleurs hyperparamètres
best_params = optimizer.max
print("Meilleurs hyperparamètres :", best_params)

# Entraîner le modèle avec les meilleurs hyperparamètres
best_model_30sec = SVC(C=best_params['params']['C'], gamma=best_params['params']['gamma'], kernel='rbf', probability=True, random_state=42)
# Training time
start_train = time.time_ns()
best_model_30sec.fit(X_train, y_train)
end_train = time.time_ns()
train_time = (end_train - start_train) / 1e9  # Convert nanoseconds to seconds
print(f"Training time: {train_time:.3f} seconds")


# Test (prediction) time
start_test = time.time_ns()
y_pred = best_model_30sec.predict(X_test)
end_test = time.time_ns()
test_time = (end_test - start_test) / 1e9  # Convert nanoseconds to seconds
print(f"Test (prediction) time: {test_time:.3f} seconds")


print("Rapport de classification :\n", classification_report(y_test, y_pred, target_names=encoder.classes_))

# Matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matrice de confusion :\n", conf_matrix)

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
| 1         | 0.69      | 37.52     | 0.09508   |
| 2         | 0.735     | 73.23     | 0.05991   |
| 3         | 0.765     | 15.69     | 0.01568   |
| 4         | 0.715     | 5.903     | 0.08663   |
| 5         | 0.73      | 60.15     | 0.07084   |
| 6         | 0.73      | 15.7      | 0.07163   |
| 7         | 0.76      | 15.67     | 0.02114   |
| 8         | 0.73      | 10.23     | 0.05221   |
| 9         | 0.765     | 15.77     | 0.0142    |
| 10        | 0.735     | 15.85     | 0.002221  |
| 11        | 0.765     | 15.7      | 0.01567   |
| 12        | 0.76      | 15.76     | 0.02105   |
| 13        | 0.77      | 15.45     | 0.01376   |
| 14        | 0.755     | 15.36     | 0.03664   |
| 15        | 0.68      | 15.48     | 0.09796   |
| 16        | 0.77      | 15.7      | 0.01648   |
| 17        | 0.755     | 15.7      | 0.03659   |
| 18        | 0.71      | 15.44     | 0.001256  |


we will continue with this dataset, now we will try logreg hoping for a better accuracy

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay
from bayes_opt import BayesianOptimization
import numpy as np
import matplotlib.pyplot as plt

# Prétraitement des données
X = df.drop(columns=['label'])  # Supprimez les colonnes non pertinentes
y = df['label']

# Encoder la variable cible
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Normaliser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Fonction d'évaluation pour BayesianOptimization
def optimize_logistic_regression(C, tol):
    model = LogisticRegression(C=C, tol=tol, max_iter=500, random_state=42)
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

# Définir les limites des hyperparamètres
param_bounds = {
    'C': (0.001, 10),
    'tol': (1e-6, 1e-2)
}

# Lancer l'optimisation
optimizer = BayesianOptimization(f=optimize_logistic_regression, pbounds=param_bounds, random_state=42)
start_opt = time.time_ns()
optimizer.maximize(init_points=5, n_iter=20)
end_opt = time.time_ns()
opt_time = (end_opt - start_opt) / 1e9  # Convert nanoseconds to seconds

print(f"Hyperparameters tuning time: {opt_time:.3f} seconds")

# Extraire les meilleurs hyperparamètres
best_params = optimizer.max
print("Meilleurs hyperparamètres :", best_params)

# Entraîner le modèle avec les meilleurs hyperparamètres
best_model = LogisticRegression(C=best_params['params']['C'], tol=best_params['params']['tol'], max_iter=500, random_state=42)
start_train = time.time_ns()
best_model.fit(X_train, y_train)
end_train = time.time_ns()
train_time = (end_train - start_train) / 1e9  # Convert nanoseconds to seconds
print(f"Training time: {train_time:.3f} seconds")
print(f"CPU usage during training: {cpu_usage_before}% -> {cpu_usage_after}%")

# Évaluation du modèle
# Test (prediction) time
start_test = time.time_ns()
y_pred = best_model.predict(X_test)
end_test = time.time_ns()
test_time = (end_test - start_test) / 1e9  # Convert nanoseconds to seconds
print(f"Test (prediction) time: {test_time:.3f} seconds")
print("Rapport de classification :\n", classification_report(y_test, y_pred, target_names=encoder.classes_))

# Matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matrice de confusion :\n", conf_matrix)

|   iter    |  target   |     C     |    tol    |
-------------------------------------------------
| 1         | 0.695     | 3.746     | 0.009507  |
| 2         | 0.71      | 7.32      | 0.005987  |
| 3         | 0.71      | 1.561     | 0.001561  |
| 4         | 0.72      | 0.5818    | 0.008662  |
| 5         | 0.705     | 6.012     | 0.007081  |
| 6         | 0.69      | 9.997     | 0.009966  |
| 7         | 0.515     | 0.001055  | 0.003255  |
| 8         | 0.71      | 1.014     | 0.005217  |
| 9         | 0.675     | 0.05743   | 0.004638  |
| 10        | 0.705     | 6.676     | 0.006422  |
| 11        | 0.72      | 0.3257    | 0.002153  |
| 12        | 0.7       | 4.419     | 0.008244  |
| 13        | 0.72      | 0.5416    | 0.008395  |
| 14        | 0.725     | 0.2745    | 0.0082    |
| 15        | 0.705     | 6.902     | 0.001225  |
| 16        | 0.705     | 6.041     | 0.001518  |
| 17        | 0.71      | 0.2093    | 0.003092  |
| 18        | 0.685     | 0.1324    | 0.006605  |


In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay
from bayes_opt import BayesianOptimization
import numpy as np
import matplotlib.pyplot as plt

# Prétraitement des données
X = df.drop(columns=['label'])  # Supprimez les colonnes non pertinentes
y = df['label']

# Encoder la variable cible
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Normaliser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Fonction d'évaluation pour BayesianOptimization
def optimize_knn(n_neighbors, p):
    model = KNeighborsClassifier(
        n_neighbors=int(n_neighbors),  # Convertir en entier
        p=int(p),                      # Distance : 1 pour Manhattan, 2 pour Euclidean
        metric='minkowski'
    )
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

# Définir les limites des hyperparamètres
param_bounds = {
    'n_neighbors': (1, 50),  # Nombre de voisins
    'p': (1, 2)             # Distance : 1 (Manhattan) ou 2 (Euclidean)
}

# Lancer l'optimisation
optimizer = BayesianOptimization(f=optimize_knn, pbounds=param_bounds, random_state=42)
start_opt = time.time_ns()
optimizer.maximize(init_points=5, n_iter=20)
end_opt = time.time_ns()
opt_time = (end_opt - start_opt) / 1e9  # Convert nanoseconds to seconds

print(f"Hyperparameters tuning time: {opt_time:.3f} seconds")

# Extraire les meilleurs hyperparamètres
best_params = optimizer.max
print("Meilleurs hyperparamètres :", best_params)

# Entraîner le modèle avec les meilleurs hyperparamètres
best_model = KNeighborsClassifier(
    n_neighbors=int(best_params['params']['n_neighbors']),
    p=int(best_params['params']['p']),
    metric='minkowski'
)
start_train = time.time_ns()
best_model.fit(X_train, y_train)
end_train = time.time_ns()
train_time = (end_train - start_train) / 1e9  # Convert nanoseconds to seconds
print(f"Training time: {train_time:.3f} seconds")

# Évaluation du modèle
cpu_usage_before = get_cpu_usage()
start_test = time.time_ns()
y_pred = best_model.predict(X_test)
end_test = time.time_ns()
cpu_usage_after = get_cpu_usage()
test_time = (end_test - start_test) / 1e9  # Convert nanoseconds to seconds
print(f"Test (prediction) time: {test_time:.3f} seconds")
print("Rapport de classification :\n", classification_report(y_test, y_pred, target_names=encoder.classes_))

# Matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matrice de confusion :\n", conf_matrix)

|   iter    |  target   | n_neig... |     p     |
-------------------------------------------------
| 1         | 0.68      | 19.35     | 1.951     |
| 2         | 0.63      | 36.87     | 1.599     |
| 3         | 0.695     | 8.645     | 1.156     |
| 4         | 0.655     | 3.846     | 1.866     |
| 5         | 0.645     | 30.45     | 1.708     |
| 6         | 0.695     | 8.629     | 1.226     |
| 7         | 0.71      | 13.11     | 1.984     |
| 8         | 0.59      | 49.98     | 1.076     |
| 9         | 0.71      | 13.59     | 1.014     |
| 10        | 0.705     | 12.04     | 1.015     |
| 11        | 0.68      | 15.34     | 1.994     |
| 12        | 0.675     | 23.98     | 1.01      |
| 13        | 0.585     | 42.79     | 1.996     |
| 14        | 0.69      | 10.64     | 1.993     |
| 15        | 0.675     | 21.16     | 1.002     |
| 16        | 0.69      | 6.753     | 1.043     |
| 17        | 0.71      | 13.01     | 1.003     |
| 18        | 0.655     | 26.7      | 1.977     |


In [31]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay
from bayes_opt import BayesianOptimization
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier


# Prétraitement des données
X = df.drop(columns=['label'])  # Supprimez les colonnes non pertinentes
y = df['label']

# Encoder la variable cible
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# No need to Normaliser les données
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
# Function for Bayesian Optimization with Random Forest
def optimize_rf(n_estimators, max_depth):
    # Convert float parameters to integers as RandomForest expects integers
    n_estimators = int(n_estimators)
    max_depth = int(max_depth)
    
    model = RandomForestClassifier(
        n_estimators=n_estimators, 
        max_depth=max_depth, 
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

# Define parameter bounds for Random Forest
param_bounds_rf = {
    'n_estimators': (10, 200),  # Number of trees
    'max_depth': (5, 50)       # Maximum depth of the trees
}

# Run Bayesian Optimization
optimizer_rf = BayesianOptimization(f=optimize_rf, pbounds=param_bounds_rf, random_state=42)
start_opt = time.time_ns()
optimizer_rf.maximize(init_points=5, n_iter=50)
end_opt = time.time_ns()
opt_time = (end_opt - start_opt) / 1e9  # Convert nanoseconds to seconds

print(f"Hyperparameters tuning time: {opt_time:.3f} seconds")
print(f"CPU usage during optimization: {cpu_usage_before}% -> {cpu_usage_after}%")

# Extract the best parameters
best_params_rf = optimizer_rf.max
print("Meilleurs hyperparamètres :", best_params_rf)

# Train the model with the best hyperparameters
best_model_rf = RandomForestClassifier(
    n_estimators=50,
    max_depth=25,
    random_state=42,
    n_jobs=-1
)
start_train = time.time_ns()
best_model_rf.fit(X_train, y_train) 
end_train = time.time_ns()
train_time = (end_train - start_train) / 1e9  # Convert nanoseconds to seconds
print(f"Training time: {train_time:.3f} seconds")

# Model evaluation
start_test = time.time_ns()
y_pred_rf = best_model_rf.predict(X_test)
end_test = time.time_ns()
test_time = (end_test - start_test) / 1e9  # Convert nanoseconds to seconds
print(f"Test (prediction) time: {test_time:.3f} seconds")
print("Rapport de classification :\n", classification_report(y_test, y_pred_rf, target_names=encoder.classes_))

# Confusion matrix
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
print("Matrice de confusion :\n", conf_matrix_rf)


|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
| 1         | 0.7133    | 21.85     | 190.6     |
| 2         | 0.7033    | 37.94     | 123.7     |
| 3         | 0.6667    | 12.02     | 39.64     |
| 4         | 0.68      | 7.614     | 174.6     |
| 5         | 0.71      | 32.05     | 144.5     |
| 6         | 0.7167    | 22.23     | 189.4     |
| 7         | 0.72      | 30.85     | 185.0     |
| 8         | 0.7167    | 33.23     | 170.2     |
| 9         | 0.71      | 48.27     | 176.6     |
| 10        | 0.7133    | 46.08     | 157.5     |
| 11        | 0.7133    | 47.04     | 199.0     |
| 12        | 0.6267    | 5.284     | 99.61     |
| 13        | 0.59      | 49.46     | 10.69     |
| 14        | 0.7       | 49.95     | 86.17     |
| 15        | 0.7067    | 49.91     | 133.7     |
| 16        | 0.7       | 49.87     | 61.17     |
| 17        | 0.71      | 49.63     | 110.2     |
| 18        | 0.63      | 5.313     | 141.3     |


In [32]:
# Import necessary libraries
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from bayes_opt import BayesianOptimization
from xgboost import XGBClassifier

X = df.drop(columns=['label'])  # Supprime la colonne cible
y = df['label']

# Encoder la variable cible
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Function for Bayesian Optimization with XGBoost
def optimize_xgb(n_estimators, max_depth, learning_rate):
    # Convert float parameters to integers where necessary
    n_estimators = int(n_estimators)
    max_depth = int(max_depth)

    model = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        objective='multi:softmax',
        eval_metric='mlogloss',
        use_label_encoder=False,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

# Define parameter bounds for XGBoost
param_bounds_xgb = {
    'n_estimators': (10, 200),
    'max_depth': (3, 20),
    'learning_rate': (0.01, 0.3)
}

# Run Bayesian Optimization
optimizer_xgb = BayesianOptimization(f=optimize_xgb, pbounds=param_bounds_xgb, random_state=42)
cpu_usage_before = get_cpu_usage()
start_opt = time.time_ns()
optimizer_xgb.maximize(init_points=5, n_iter=50)
end_opt = time.time_ns()
opt_time = (end_opt - start_opt) / 1e9  # Convert nanoseconds to seconds

print(f"Hyperparameters tuning time: {opt_time:.3f} seconds")
print(f"CPU usage during optimization: {cpu_usage_before}% -> {cpu_usage_after}%")

# Extract the best parameters
best_params_xgb = optimizer_xgb.max['params']
best_params_xgb['n_estimators'] = int(best_params_xgb['n_estimators'])
best_params_xgb['max_depth'] = int(best_params_xgb['max_depth'])
print("Meilleurs hyperparamètres :", best_params_xgb)

# Train the model with the best hyperparameters
best_model_xgb = XGBClassifier(
    n_estimators=best_params_xgb['n_estimators'],
    max_depth=best_params_xgb['max_depth'],
    learning_rate=best_params_xgb['learning_rate'],
    objective='multi:softmax',
    eval_metric='mlogloss',
    use_label_encoder=False,
    random_state=42,
    n_jobs=-1
)

start_train = time.time_ns()
best_model_xgb.fit(X_train, y_train)
end_train = time.time_ns()
train_time = (end_train - start_train) / 1e9  # Convert nanoseconds to seconds
print(f"Training time: {train_time:.3f} seconds")

# Model evaluation
start_test = time.time_ns()
y_pred_xgb = best_model_xgb.predict(X_test)
end_test = time.time_ns()
test_time = (end_test - start_test) / 1e9  # Convert nanoseconds to seconds
print(f"Test (prediction) time: {test_time:.3f} seconds")

# Rapport de classification
print("Rapport de classification :\n", classification_report(y_test, y_pred_xgb, target_names=encoder.classes_))

# Confusion matrix
conf_matrix_xgb = confusion_matrix(y_test, y_pred_xgb)
print("Matrice de confusion :\n", conf_matrix_xgb)


|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
| 1         | 0.7233    | 0.1186    | 19.16     | 149.1     |
| 2         | 0.7033    | 0.1836    | 5.652     | 39.64     |
| 3         | 0.6667    | 0.02684   | 17.72     | 124.2     |
| 4         | 0.7267    | 0.2153    | 3.35      | 194.3     |
| 5         | 0.7       | 0.2514    | 6.61      | 44.55     |
| 6         | 0.71      | 0.1483    | 9.617     | 178.6     |
| 7         | 0.6833    | 0.2923    | 16.54     | 199.9     |
| 8         | 0.6667    | 0.01609   | 3.118     | 187.0     |
| 9         | 0.7233    | 0.1563    | 3.33      | 193.2     |
| 10        | 0.6933    | 0.02991   | 7.398     | 195.2     |
| 11        | 0.7033    | 0.1669    | 15.69     | 149.6     |
| 12        | 0.7167    | 0.2297    | 19.96     | 145.9     |
| 13        | 0.7167    | 0.2161    | 19.67     | 153.1     |
| 14        | 0.73      | 0.2621    | 3.167     | 198.9     |
| 15    

form here onward data 3s

In [33]:
df = pd.read_csv("/kaggle/input/gtzan-dataset-music-genre-classification/Data/features_3_sec.csv")

In [34]:
df.drop(columns=['filename', 'length'], inplace=True)

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay
from bayes_opt import BayesianOptimization
import numpy as np
import matplotlib.pyplot as plt

# Prétraitement des données
X = df.drop(columns=['label'])  # Supprimez les colonnes non pertinentes
y = df['label']

# Encoder la variable cible
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Normaliser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Fonction d'évaluation pour BayesianOptimization
def optimize_svm(C, gamma):
    model = SVC(C=C, gamma=gamma, kernel='rbf', probability=True, random_state=42)
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

# Définir les limites des hyperparamètres
param_bounds = {
    'C': (0.1, 100),      # Paramètre de régularisation
    'gamma': (1e-4, 1e-1) # Coefficient pour le noyau RBF
}

# Lancer l'optimisation
optimizer = BayesianOptimization(f=optimize_svm, pbounds=param_bounds, random_state=42)
start_opt = time.time_ns()
optimizer.maximize(init_points=5, n_iter=20)
end_opt = time.time_ns()
opt_time = (end_opt - start_opt) / 1e9  # Convert nanoseconds to seconds

print(f"Hyperparameters tuning time: {opt_time:.3f} seconds")


# Extraire les meilleurs hyperparamètres
best_params = optimizer.max
print("Meilleurs hyperparamètres :", best_params)

# Entraîner le modèle avec les meilleurs hyperparamètres
best_model_3sec = SVC(C=best_params['params']['C'], gamma=best_params['params']['gamma'], kernel='rbf', probability=True, random_state=42)
# Training time
start_train = time.time_ns()
best_model_3sec.fit(X_train, y_train)
end_train = time.time_ns()
train_time = (end_train - start_train) / 1e9  # Convert nanoseconds to seconds
print(f"Training time: {train_time:.3f} seconds")


# Test (prediction) time
start_test = time.time_ns()
y_pred = best_model_3sec.predict(X_test)
end_test = time.time_ns()
test_time = (end_test - start_test) / 1e9  # Convert nanoseconds to seconds
print(f"Test (prediction) time: {test_time:.3f} seconds")


print("Rapport de classification :\n", classification_report(y_test, y_pred, target_names=encoder.classes_))

# Matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matrice de confusion :\n", conf_matrix)

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
| 1         | 0.8699    | 37.52     | 0.09508   |
| 2         | 0.9044    | 73.23     | 0.05991   |
| 3         | 0.9129    | 15.69     | 0.01568   |
| 4         | 0.8789    | 5.903     | 0.08663   |
| 5         | 0.8934    | 60.15     | 0.07084   |
| 6         | 0.8914    | 15.72     | 0.07382   |
| 7         | 0.9159    | 82.86     | 0.0301    |
| 8         | 0.9119    | 10.23     | 0.05221   |
| 9         | 0.8649    | 0.6638    | 0.04643   |
| 10        | 0.9024    | 66.79     | 0.06425   |
| 11        | 0.9074    | 3.344     | 0.0216    |
| 12        | 0.8814    | 44.24     | 0.08245   |
| 13        | 0.9109    | 1.96      | 0.04389   |
| 14        | 0.9134    | 39.57     | 0.04879   |
| 15        | 0.9089    | 69.05     | 0.01233   |
| 16        | 0.9134    | 60.45     | 0.01525   |
| 17        | 0.8774    | 79.3      | 0.08841   |
| 18        | 0.9124    | 42.0      | 0.01663   |


In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay
from bayes_opt import BayesianOptimization
import numpy as np
import matplotlib.pyplot as plt

# Prétraitement des données
X = df.drop(columns=['label'])  # Supprimez les colonnes non pertinentes
y = df['label']

# Encoder la variable cible
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Normaliser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Fonction d'évaluation pour BayesianOptimization
def optimize_logistic_regression(C, tol):
    model = LogisticRegression(C=C, tol=tol, max_iter=500, random_state=42)
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

# Définir les limites des hyperparamètres
param_bounds = {
    'C': (0.001, 10),
    'tol': (1e-6, 1e-2)
}

# Lancer l'optimisation
optimizer = BayesianOptimization(f=optimize_logistic_regression, pbounds=param_bounds, random_state=42)
start_opt = time.time_ns()
optimizer.maximize(init_points=5, n_iter=20)
end_opt = time.time_ns()
opt_time = (end_opt - start_opt) / 1e9  # Convert nanoseconds to seconds

print(f"Hyperparameters tuning time: {opt_time:.3f} seconds")

# Extraire les meilleurs hyperparamètres
best_params = optimizer.max
print("Meilleurs hyperparamètres :", best_params)

# Entraîner le modèle avec les meilleurs hyperparamètres
best_model = LogisticRegression(C=best_params['params']['C'], tol=best_params['params']['tol'], max_iter=500, random_state=42)
start_train = time.time_ns()
best_model.fit(X_train, y_train)
end_train = time.time_ns()
train_time = (end_train - start_train) / 1e9  # Convert nanoseconds to seconds
print(f"Training time: {train_time:.3f} seconds")
print(f"CPU usage during training: {cpu_usage_before}% -> {cpu_usage_after}%")

# Évaluation du modèle
# Test (prediction) time
start_test = time.time_ns()
y_pred = best_model.predict(X_test)
end_test = time.time_ns()
test_time = (end_test - start_test) / 1e9  # Convert nanoseconds to seconds
print(f"Test (prediction) time: {test_time:.3f} seconds")
print(f"CPU usage during test: {cpu_usage_before}% -> {cpu_usage_after}%")
print("Rapport de classification :\n", classification_report(y_test, y_pred, target_names=encoder.classes_))

# Matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matrice de confusion :\n", conf_matrix)

|   iter    |  target   |     C     |    tol    |
-------------------------------------------------
| 1         | 0.7217    | 3.746     | 0.009507  |


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


| 2         | 0.7242    | 7.32      | 0.005987  |
| 3         | 0.7217    | 1.561     | 0.001561  |
| 4         | 0.7127    | 0.5818    | 0.008662  |
| 5         | 0.7222    | 6.012     | 0.007081  |


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


| 6         | 0.7227    | 6.73      | 0.005309  |


...

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


| 22        | 0.7237    | 7.067     | 0.009274  |
| 23        | 0.7242    | 7.932     | 0.009595  |


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


| 24        | 0.7242    | 9.781     | 0.001201  |
| 25        | 0.7222    | 6.363     | 0.009616  |
Hyperparameters tuning time: 31.588 seconds
Meilleurs hyperparamètres : {'target': 0.7247247247247247, 'params': {'C': 7.7335010248040055, 'tol': 0.009751074741538348}}
Training time: 1.272 seconds
CPU usage during training: 0.5% -> 0.2%
Test (prediction) time: 0.002 seconds
CPU usage during test: 0.5% -> 0.2%
Rapport de classification :
               precision    recall  f1-score   support

       blues       0.67      0.72      0.70       200
   classical       0.91      0.95      0.93       199
     country       0.63      0.59      0.61       199
       disco       0.66      0.67      0.66       200
      hiphop       0.76      0.61      0.68       200
        jazz       0.80      0.85      0.83       200
       metal       0.82      0.86      0.84       200
         pop       0.79      0.77      0.78       200
      reggae       0.66      0.69      0.67       200
        rock      

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay
from bayes_opt import BayesianOptimization
import numpy as np
import matplotlib.pyplot as plt

# Prétraitement des données
X = df.drop(columns=['label'])  # Supprimez les colonnes non pertinentes
y = df['label']

# Encoder la variable cible
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Normaliser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Fonction d'évaluation pour BayesianOptimization
def optimize_knn(n_neighbors, p):
    model = KNeighborsClassifier(
        n_neighbors=int(n_neighbors),  # Convertir en entier
        p=int(p),                      # Distance : 1 pour Manhattan, 2 pour Euclidean
        metric='minkowski'
    )
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

# Définir les limites des hyperparamètres
param_bounds = {
    'n_neighbors': (1, 50),  # Nombre de voisins
    'p': (1, 2)             # Distance : 1 (Manhattan) ou 2 (Euclidean)
}

# Lancer l'optimisation
optimizer = BayesianOptimization(f=optimize_knn, pbounds=param_bounds, random_state=42)
start_opt = time.time_ns()
optimizer.maximize(init_points=5, n_iter=20)
end_opt = time.time_ns()
opt_time = (end_opt - start_opt) / 1e9  # Convert nanoseconds to seconds

print(f"Hyperparameters tuning time: {opt_time:.3f} seconds")
print(f"CPU usage during optimization: {cpu_usage_before}% -> {cpu_usage_after}%")

# Extraire les meilleurs hyperparamètres
best_params = optimizer.max
print("Meilleurs hyperparamètres :", best_params)

# Entraîner le modèle avec les meilleurs hyperparamètres
best_model_kmeans = KNeighborsClassifier(
    n_neighbors=int(best_params['params']['n_neighbors']),
    p=int(best_params['params']['p']),
    metric='minkowski'
)
start_train = time.time_ns()
best_model_kmeans.fit(X_train, y_train)
end_train = time.time_ns()
train_time = (end_train - start_train) / 1e9  # Convert nanoseconds to seconds
print(f"Training time: {train_time:.3f} seconds")

# Évaluation du modèle
start_test = time.time_ns()
y_pred = best_model_kmeans.predict(X_test)
end_test = time.time_ns()
test_time = (end_test - start_test) / 1e9  # Convert nanoseconds to seconds
print(f"Test (prediction) time: {test_time:.3f} seconds")
print("Rapport de classification :\n", classification_report(y_test, y_pred, target_names=encoder.classes_))

# Matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matrice de confusion :\n", conf_matrix)

|   iter    |  target   | n_neig... |     p     |
-------------------------------------------------
| 1         | 0.8293    | 19.35     | 1.951     |
| 2         | 0.7808    | 36.87     | 1.599     |
| 3         | 0.8859    | 8.645     | 1.156     |
| 4         | 0.9164    | 3.846     | 1.866     |
| 5         | 0.7923    | 30.45     | 1.708     |
| 6         | 0.9214    | 1.012     | 1.127     |
| 7         | 0.9214    | 1.11      | 1.997     |
| 8         | 0.9214    | 1.938     | 1.021     |
| 9         | 0.9214    | 1.673     | 1.599     |
| 10        | 0.9214    | 1.546     | 1.017     |
| 11        | 0.9214    | 1.028     | 1.549     |
| 12        | 0.9214    | 1.412     | 1.437     |
| 13        | 0.9214    | 1.442     | 1.059     |
| 14        | 0.9214    | 1.54      | 1.98      |
| 15        | 0.9214    | 1.445     | 1.007     |
| 16        | 0.9214    | 1.014     | 1.947     |
| 17        | 0.9214    | 1.392     | 1.549     |
| 18        | 0.9214    | 1.001     | 1.014     |


In [38]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay
from bayes_opt import BayesianOptimization
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier


# Prétraitement des données
X = df.drop(columns=['label'])  # Supprimez les colonnes non pertinentes
y = df['label']

# Encoder la variable cible
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# No need to Normaliser les données
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
# Function for Bayesian Optimization with Random Forest
def optimize_rf(n_estimators, max_depth):
    # Convert float parameters to integers as RandomForest expects integers
    n_estimators = int(n_estimators)
    max_depth = int(max_depth)
    
    model = RandomForestClassifier(
        n_estimators=n_estimators, 
        max_depth=max_depth, 
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

# Define parameter bounds for Random Forest
param_bounds_rf = {
    'n_estimators': (10, 200),  # Number of trees
    'max_depth': (5, 50)       # Maximum depth of the trees
}

# Run Bayesian Optimization
optimizer_rf = BayesianOptimization(f=optimize_rf, pbounds=param_bounds_rf, random_state=42)
start_opt = time.time_ns()
optimizer_rf.maximize(init_points=5, n_iter=50)
end_opt = time.time_ns()
opt_time = (end_opt - start_opt) / 1e9  # Convert nanoseconds to seconds

print(f"Hyperparameters tuning time: {opt_time:.3f} seconds")

# Extract the best parameters
best_params_rf = optimizer_rf.max
print("Meilleurs hyperparamètres :", best_params_rf)

# Train the model with the best hyperparameters
best_model_rf = RandomForestClassifier(
    n_estimators=50,
    max_depth=25,
    random_state=42,
    n_jobs=-1
)
start_train = time.time_ns()
best_model_rf.fit(X_train, y_train) 
end_train = time.time_ns()
train_time = (end_train - start_train) / 1e9  # Convert nanoseconds to seconds
print(f"Training time: {train_time:.3f} seconds")

# Model evaluation
start_test = time.time_ns()
y_pred_rf = best_model_rf.predict(X_test)
end_test = time.time_ns()
test_time = (end_test - start_test) / 1e9  # Convert nanoseconds to seconds
print(f"Test (prediction) time: {test_time:.3f} seconds")
print("Rapport de classification :\n", classification_report(y_test, y_pred_rf, target_names=encoder.classes_))

# Confusion matrix
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
print("Matrice de confusion :\n", conf_matrix_rf)


|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
| 1         | 0.8575    | 21.85     | 190.6     |
| 2         | 0.8582    | 37.94     | 123.7     |
| 3         | 0.8055    | 12.02     | 39.64     |
| 4         | 0.6984    | 7.614     | 174.6     |
| 5         | 0.8569    | 32.05     | 144.5     |
| 6         | 0.8595    | 39.0      | 125.0     |
| 7         | 0.8602    | 37.64     | 199.3     |
| 8         | 0.8582    | 46.59     | 178.3     |
| 9         | 0.8602    | 49.87     | 153.1     |
| 10        | 0.8535    | 49.94     | 94.71     |
| 11        | 0.8482    | 21.26     | 92.68     |
| 12        | 0.8478    | 49.67     | 68.0      |
| 13        | 0.5986    | 5.725     | 118.3     |
| 14        | 0.8582    | 23.33     | 69.89     |
| 15        | 0.7484    | 49.46     | 10.93     |
| 16        | 0.5609    | 5.479     | 10.83     |
| 17        | 0.5953    | 5.074     | 64.75     |
| 18        | 0.8482    | 35.61     | 80.46     |


In [39]:
# Import necessary libraries
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from bayes_opt import BayesianOptimization
from xgboost import XGBClassifier

X = df.drop(columns=['label'])  # Supprime la colonne cible
y = df['label']

# Encoder la variable cible
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Function for Bayesian Optimization with XGBoost
def optimize_xgb(n_estimators, max_depth, learning_rate):
    # Convert float parameters to integers where necessary
    n_estimators = int(n_estimators)
    max_depth = int(max_depth)

    model = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        objective='multi:softmax',
        eval_metric='mlogloss',
        use_label_encoder=False,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

# Define parameter bounds for XGBoost
param_bounds_xgb = {
    'n_estimators': (10, 200),
    'max_depth': (3, 20),
    'learning_rate': (0.01, 0.3)
}

# Run Bayesian Optimization
optimizer_xgb = BayesianOptimization(f=optimize_xgb, pbounds=param_bounds_xgb, random_state=42)
cpu_usage_before = get_cpu_usage()
start_opt = time.time_ns()
optimizer_xgb.maximize(init_points=5, n_iter=50)
end_opt = time.time_ns()
opt_time = (end_opt - start_opt) / 1e9  # Convert nanoseconds to seconds

print(f"Hyperparameters tuning time: {opt_time:.3f} seconds")
print(f"CPU usage during optimization: {cpu_usage_before}% -> {cpu_usage_after}%")

# Extract the best parameters
best_params_xgb = optimizer_xgb.max['params']
best_params_xgb['n_estimators'] = int(best_params_xgb['n_estimators'])
best_params_xgb['max_depth'] = int(best_params_xgb['max_depth'])
print("Meilleurs hyperparamètres :", best_params_xgb)

# Train the model with the best hyperparameters
best_model_xgb = XGBClassifier(
    n_estimators=best_params_xgb['n_estimators'],
    max_depth=best_params_xgb['max_depth'],
    learning_rate=best_params_xgb['learning_rate'],
    objective='multi:softmax',
    eval_metric='mlogloss',
    use_label_encoder=False,
    random_state=42,
    n_jobs=-1
)

start_train = time.time_ns()
best_model_xgb.fit(X_train, y_train)
end_train = time.time_ns()
train_time = (end_train - start_train) / 1e9  # Convert nanoseconds to seconds
print(f"Training time: {train_time:.3f} seconds")

# Model evaluation
start_test = time.time_ns()
y_pred_xgb = best_model_xgb.predict(X_test)
end_test = time.time_ns()
test_time = (end_test - start_test) / 1e9  # Convert nanoseconds to seconds
print(f"Test (prediction) time: {test_time:.3f} seconds")

# Rapport de classification
print("Rapport de classification :\n", classification_report(y_test, y_pred_xgb, target_names=encoder.classes_))

# Confusion matrix
conf_matrix_xgb = confusion_matrix(y_test, y_pred_xgb)
print("Matrice de confusion :\n", conf_matrix_xgb)


|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
| 1         | 0.8802    | 0.1186    | 19.16     | 149.1     |
| 2         | 0.8272    | 0.1836    | 5.652     | 39.64     |
| 3         | 0.8392    | 0.02684   | 17.72     | 124.2     |
| 4         | 0.8685    | 0.2153    | 3.35      | 194.3     |
| 5         | 0.8629    | 0.2514    | 6.61      | 44.55     |
| 6         | 0.8699    | 0.2369    | 3.392     | 194.2     |
| 7         | 0.8629    | 0.06187   | 17.48     | 150.5     |
| 8         | 0.8805    | 0.1463    | 18.11     | 147.2     |
| 9         | 0.8932    | 0.2571    | 4.503     | 191.0     |
| 10        | 0.8916    | 0.258     | 8.046     | 191.5     |
| 11        | 0.8785    | 0.08878   | 6.733     | 186.8     |
| 12        | 0.8802    | 0.09182   | 13.39     | 191.7     |
| 13        | 0.8829    | 0.2844    | 10.82     | 196.2     |
| 14        | 0.8622    | 0.05479   | 19.9      | 141.4     |
| 15    

In [40]:
print(f"Utilisation maximale des ressources pendant l'exécution :")
print(f"CPU : {max_cpu_usage:.2f} %")
print(f"RAM : {max_ram_usage:.2f} MB")

Utilisation maximale des ressources pendant l'exécution :
CPU : 100.00 %
RAM : 1223.62 MB


we will use the best model (accuracy and precision wise) that was trained on the 30 sec and the best one that was trained on 3sec to test generalization

In [41]:
import requests 
local_file_path = "/kaggle/working/pop_audio.wav"
response = requests.get('https://benjamin9.oceansaver.in/pacific/?LUEdC2F7gKMZUjyM59uSlw5')
with open(local_file_path, "wb") as file:
        file.write(response.content)

In [42]:
from pydub import AudioSegment
import librosa
import numpy as np
import os
import pandas as pd


def repeat_audio_to_30s(audio, target_length_ms=30000):
    while len(audio) < target_length_ms:
        audio += audio 
    return audio[:target_length_ms]



# Function to split audio into 30-second chunks
def split_audio(input_file, chunk_length_ms=30000):
    if input_file.lower().endswith('.mp3'):
        audio = AudioSegment.from_mp3(input_file)
    elif input_file.lower().endswith('.wav'):
        audio = AudioSegment.from_wav(input_file)
    else:
        raise ValueError("Format de fichier non pris en charge. Utilisez .mp3 ou .wav")

    if len(audio) < chunk_length_ms:
        audio = repeat_audio_to_30s(audio, chunk_length_ms)
    
    if len(audio) < chunk_length_ms:
        audio = repeat_audio_to_30s(audio, chunk_length_ms)
    
    chunks = []
    preds = []
    i = 0
    
    for start_ms in range(0, len(audio), chunk_length_ms):
        chunk = audio[start_ms:start_ms + chunk_length_ms]

        if len(chunk) < chunk_length_ms:
            chunk = repeat_audio_to_30s(chunk, chunk_length_ms)
        
        chunks.append(chunk)
    return chunks

# Function to extract features from an audio chunk
def extract_features(audio_chunk, sr=22050, n_mfcc=20):
    # Export the chunk to a temporary file
    chunk_path = "temp_chunk.wav"
    audio_chunk.export(chunk_path, format="wav")
    
    # Load the audio using librosa
    y, sr = librosa.load(chunk_path, sr=sr)
    
    # Extract features
    features = {}
    
    # Length of the audio file
    features['length'] = len(y) / sr
    
    # Chroma features
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    features['chroma_stft_mean'] = np.mean(chroma_stft)
    features['chroma_stft_var'] = np.var(chroma_stft)
    
    # RMS (Root Mean Square)
    rms = librosa.feature.rms(y=y)
    features['rms_mean'] = np.mean(rms)
    features['rms_var'] = np.var(rms)
    
    # Spectral Centroid
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    features['spectral_centroid_mean'] = np.mean(spectral_centroid)
    features['spectral_centroid_var'] = np.var(spectral_centroid)
    
    # Spectral Bandwidth
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    features['spectral_bandwidth_mean'] = np.mean(spectral_bandwidth)
    features['spectral_bandwidth_var'] = np.var(spectral_bandwidth)
    
    # Spectral Rolloff
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr, roll_percent=0.85)
    features['rolloff_mean'] = np.mean(rolloff)
    features['rolloff_var'] = np.var(rolloff)
    
    # Zero Crossing Rate
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=y)
    features['zero_crossing_rate_mean'] = np.mean(zero_crossing_rate)
    features['zero_crossing_rate_var'] = np.var(zero_crossing_rate)
    
    # Harmony
    harmony = librosa.effects.harmonic(y)
    features['harmony_mean'] = np.mean(harmony)
    features['harmony_var'] = np.var(harmony)
    
    # Perceptual sharpness (perceptr)
    perceptr = librosa.feature.spectral_flatness(y=y)
    features['perceptr_mean'] = np.mean(perceptr)
    features['perceptr_var'] = np.var(perceptr)
    
    # Tempo
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo, _ = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)
    features['tempo'] = tempo
    
    # MFCCs (Mean and Variance for each coefficient)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    for i in range(n_mfcc):
        features[f'mfcc{i+1}_mean'] = np.mean(mfcc[i])
        features[f'mfcc{i+1}_var'] = np.var(mfcc[i])
    
    # Remove the temporary file
    os.remove(chunk_path)
    
    return features


# Main function to process the audio and extract features for each chunk
def process_audio(input_file):
    # Split the audio into 30-second chunks
    chunks = split_audio(input_file)
    
    # Prepare list to collect all features
    all_features = []
    
    # Extract features for each chunk
    for i, chunk in enumerate(chunks):
        features = extract_features(chunk)
        features['filename'] = os.path.basename(input_file)  # Add the filename
        all_features.append(features)
        print(f"Processed chunk {i+1}/{len(chunks)}")
        print(chunk)
    
    # Convert the features to a pandas DataFrame
    df = pd.DataFrame(all_features)
    
    return df

# Example usage
input_file = '/kaggle/input/dataset10/hiphop.mp3'  # Path to the input audio file
features_df = process_audio(input_file)

# Save the features DataFrame to a CSV file for future use
features_df.to_csv('audio_features.csv', index=False)

# Display the extracted features for the first chunk (optional)
print(features_df.head())


Processed chunk 1/9
Processed chunk 2/9
Processed chunk 3/9
Processed chunk 4/9
Processed chunk 5/9
Processed chunk 6/9
Processed chunk 7/9
Processed chunk 8/9
Processed chunk 9/9
   length  chroma_stft_mean  chroma_stft_var  rms_mean   rms_var  \
0    30.0          0.448440         0.093858  0.245124  0.025980   
1    30.0          0.431771         0.092006  0.266856  0.016434   
2    30.0          0.435785         0.090056  0.264571  0.015482   
3    30.0          0.468479         0.089454  0.255062  0.017763   
4    30.0          0.427247         0.092445  0.254092  0.017834   

   spectral_centroid_mean  spectral_centroid_var  spectral_bandwidth_mean  \
0             1794.458998           1.130024e+06              2135.965908   
1             2085.637871           1.104482e+06              2394.809097   
2             2155.580028           1.119819e+06              2427.754302   
3             2246.575363           1.436674e+06              2460.140933   
4             2242.958703 

In [43]:
X_test_audio = features_df

In [44]:
X_test_audio.drop(columns=['filename', 'length'], inplace=True)

In [45]:
scaler = StandardScaler()
X_test_audio = scaler.fit_transform(X_test_audio)

we chose the svm model with .78 accuracy that was trained on 30s audios

In [46]:
y_pred = best_model_30sec.predict(X_test_audio)

In [47]:
from collections import Counter

Counter(y_pred)

Counter({0: 1, 2: 4, 3: 1, 9: 1, 6: 1, 1: 1})

In [48]:
len(y_pred)

9

we will use kmeans model with .92 that was trained on 3s audios

In [50]:
from pydub import AudioSegment
import librosa
import numpy as np
import os
import pandas as pd

# Function to split audio into 3-second chunks
def split_audio(input_file, chunk_length_ms=3000):
    if input_file.lower().endswith('.mp3'):
        audio = AudioSegment.from_mp3(input_file)
    elif input_file.lower().endswith('.wav'):
        audio = AudioSegment.from_wav(input_file)
    else:
        raise ValueError("Format de fichier non pris en charge. Utilisez .mp3 ou .wav")

    if len(audio) < chunk_length_ms:
        audio = repeat_audio_to_30s(audio, chunk_length_ms)
    
    if len(audio) < chunk_length_ms:
        audio = repeat_audio_to_30s(audio, chunk_length_ms)
    
    chunks = []
    preds = []
    i = 0
    
    for start_ms in range(0, len(audio), chunk_length_ms):
        chunk = audio[start_ms:start_ms + chunk_length_ms]

        if len(chunk) < chunk_length_ms:
            chunk = repeat_audio_to_30s(chunk, chunk_length_ms)
        
        chunks.append(chunk)
    return chunks

# Function to extract features from an audio chunk
def extract_features(audio_chunk, sr=22050, n_mfcc=20):
    # Export the chunk to a temporary file
    chunk_path = "temp_chunk.wav"
    audio_chunk.export(chunk_path, format="wav")
    
    # Load the audio using librosa
    y, sr = librosa.load(chunk_path, sr=sr)
    
    # Extract features
    features = {}
    
    # Length of the audio file
    features['length'] = len(y) / sr
    
    # Chroma features
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    features['chroma_stft_mean'] = np.mean(chroma_stft)
    features['chroma_stft_var'] = np.var(chroma_stft)
    
    # RMS (Root Mean Square)
    rms = librosa.feature.rms(y=y)
    features['rms_mean'] = np.mean(rms)
    features['rms_var'] = np.var(rms)
    
    # Spectral Centroid
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    features['spectral_centroid_mean'] = np.mean(spectral_centroid)
    features['spectral_centroid_var'] = np.var(spectral_centroid)
    
    # Spectral Bandwidth
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    features['spectral_bandwidth_mean'] = np.mean(spectral_bandwidth)
    features['spectral_bandwidth_var'] = np.var(spectral_bandwidth)
    
    # Spectral Rolloff
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr, roll_percent=0.85)
    features['rolloff_mean'] = np.mean(rolloff)
    features['rolloff_var'] = np.var(rolloff)
    
    # Zero Crossing Rate
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=y)
    features['zero_crossing_rate_mean'] = np.mean(zero_crossing_rate)
    features['zero_crossing_rate_var'] = np.var(zero_crossing_rate)
    
    # Harmony
    harmony = librosa.effects.harmonic(y)
    features['harmony_mean'] = np.mean(harmony)
    features['harmony_var'] = np.var(harmony)
    
    # Perceptual sharpness (perceptr)
    perceptr = librosa.feature.spectral_flatness(y=y)
    features['perceptr_mean'] = np.mean(perceptr)
    features['perceptr_var'] = np.var(perceptr)
    
    # Tempo
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo, _ = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)
    features['tempo'] = tempo
    
    # MFCCs (Mean and Variance for each coefficient)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    for i in range(n_mfcc):
        features[f'mfcc{i+1}_mean'] = np.mean(mfcc[i])
        features[f'mfcc{i+1}_var'] = np.var(mfcc[i])
    
    # Remove the temporary file
    os.remove(chunk_path)
    
    return features

# Main function to process the audio and extract features for each chunk
def process_audio(input_file):
    # Split the audio into 30-second chunks
    chunks = split_audio(input_file)
    
    # Prepare list to collect all features
    all_features = []
    
    # Extract features for each chunk
    for i, chunk in enumerate(chunks):
        features = extract_features(chunk)
        features['filename'] = os.path.basename(input_file)  # Add the filename
        all_features.append(features)
        print(f"Processed chunk {i+1}/{len(chunks)}")
        print(chunk)
    
    # Convert the features to a pandas DataFrame
    df = pd.DataFrame(all_features)
    
    return df

# Example usage
input_file = '/kaggle/input/dataset10/hiphop.mp3'  # Path to the input audio file
features_df = process_audio(input_file)

# Save the features DataFrame to a CSV file for future use
features_df.to_csv('audio_features.csv', index=False)

# Display the extracted features for the first chunk (optional)
print(features_df.head())


Processed chunk 1/90
Processed chunk 2/90
Processed chunk 3/90
Processed chunk 4/90
Processed chunk 5/90
Processed chunk 6/90
Processed chunk 7/90
Processed chunk 8/90
Processed chunk 9/90
Processed chunk 10/90
Processed chunk 11/90
Processed chunk 12/90
Processed chunk 13/90
Processed chunk 14/90
Processed chunk 15/90
Processed chunk 16/90
Processed chunk 17/90
Processed chunk 18/90
Processed chunk 19/90
Processed chunk 20/90
Processed chunk 21/90
Processed chunk 22/90
Processed chunk 23/90
Processed chunk 24/90
Processed chunk 25/90
Processed chunk 26/90
Processed chunk 27/90
Processed chunk 28/90
Processed chunk 29/90
Processed chunk 30/90
Processed chunk 31/90
Processed chunk 32/90
Processed chunk 33/90
Processed chunk 34/90
Processed chunk 35/90
Processed chunk 36/90
Processed chunk 37/90
Processed chunk 38/90
Processed chunk 39/90
Processed chunk 40/90
Processed chunk 41/90
Processed chunk 42/90
Processed chunk 43/90
Processed chunk 44/90
Processed chunk 45/90
Processed chunk 46/

In [52]:
X_test_audio = features_df

In [53]:
X_test_audio.drop(columns=['filename', 'length'], inplace=True)

In [54]:
scaler = StandardScaler()
X_test_audio = scaler.fit_transform(X_test_audio)

In [55]:
pred_kmeans = best_model_kmeans.predict(X_test_audio)

In [56]:
from collections import Counter

Counter(pred_kmeans)

Counter({9: 13, 0: 9, 4: 7, 2: 19, 3: 19, 7: 3, 8: 8, 1: 4, 6: 7, 5: 1})